# -------------------软采小区完整性评估----------------------
###   ：使用亿阳导出的软采小区列表与全网小区数据对比，计算软采采集率

In [1]:
%load_ext autoreload
%autoreload 2shange

In [2]:
import pandas as pd
import geopandas as gpd
import mytools

## 模块1、导入软采数据

In [3]:
file_name = 'G:/1-规划/MRO/1、原始数据/摸底测试相关/2020年/4月/软采/allcgi04/allcgi04.csv'

In [12]:
ruancai_cel = pd.read_csv(open(file_name,encoding='utf-8'),sep='\t')

In [21]:
ruancai_cel_use = ruancai_cel.sort_values('rsrp_sample_count',ascending=False)

In [23]:
ruancai_cel_use2 = ruancai_cel_use.drop_duplicates('eutrancell_cgi')

In [24]:
ruancai_cel_use2.shape

(331256, 13)

In [37]:
ruancai_cel_use2.to_clipboard()

In [26]:
lie = ['day', 'city', 'eutrancell_cgi', 'eutrancell_name','rsrp_sample_count', 
    'rsrp_weak', 'a.un_maxrsrp_sample_count', 'a.un_maxrsrp_weak_l2', 'a.te_maxrsrp_sample_count', 'a.te_maxrsrp_weak_l2','hd','总问题小区']

In [29]:
ruancai_cel =ruancai_cel_use2.reindex(columns=lie)

In [30]:
day,city,cgi,cell_name,rsrp_count,rsrp_weak,un_count,un_weak,te_count,te_weak ,hd,wt_z= lie

In [31]:
ruancai_cel_use  = ruancai_cel[[day,city,cgi,cell_name,rsrp_count,rsrp_weak,un_count,un_weak,te_count,te_weak,hd,wt_z]]

###  )替换竞对数据__

In [ ]:
con = mytools.othern.con()

jingdui = pd.read_sql_table(table_name='多天指标汇总-mro-19年9月竞对4天使用月底',con=con)

jingdui_use = jingdui[[ 'cgi',  'un_maxrsrp_count', 'un_maxrsrp_weak113','te_maxrsrp_count', 'te_maxrsrp_weak113']]
jingdui_use.columns = ['cgi','a.un_maxrsrp_sample_count', 'a.un_maxrsrp_weak_l2', 'a.te_maxrsrp_sample_count', 'a.te_maxrsrp_weak_l2']

ruancai_cgi = ruancai_cel_use.set_index('cgi')
jingdui_cgi = jingdui_use.set_index('cgi')

ruancai_cgi.update(jingdui_cgi)

In [ ]:
ruancai_cgi = ruancai_cgi.reset_index()

In [ ]:
mytools.othern.pickle_save(ruancai_cgi,'G:/1-规划/评估/软采评估/2019第四季度/软采小区/使用摸底小区/存放数据/ruancai_jingdui.data')

## 模块2、处理数据

In [34]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20200108.csv',encoding='gbk'),usecols=['小区CGI(*)','所属地区'])

In [35]:
gongcan_cgi,gongcan_city=['小区CGI(*)','所属地区']

In [36]:
#定义结果的列
gongcan_cgi_sum,cgi_sum,un_valid_sum,te_valid_sum,lv_cmcc,lv_un,lv_te=[
    '现网小区数量', '软采数量', '联通有效小区', '电信有效小区','软采采集率_本网', '软采小区采集率_联通', '软采小区采集率_电信']

In [ ]:
gongcan_merge = gongcan.merge(ruancai_cgi,how='left',left_on=gongcan_cgi,right_on=cgi)

gongcan_merge[un_count] = pd.to_numeric(gongcan_merge[un_count])
gongcan_merge[te_count] = pd.to_numeric(gongcan_merge[te_count])
cmcc_data = gongcan_merge.groupby(gongcan_city).agg({gongcan_cgi:'count',cgi:'count'})
un_data = gongcan_merge.loc[gongcan_merge[un_count]>1000].groupby(gongcan_city).agg({gongcan_cgi:'count'})
te_data = gongcan_merge.loc[gongcan_merge[te_count]>1000].groupby(gongcan_city).agg({gongcan_cgi:'count'})

jingdui_data = un_data.merge(te_data,left_index=True,right_index=True,suffixes=('_un','_te'))
data_res = cmcc_data.merge(jingdui_data,how='left',left_index=True,right_index=True,suffixes=('_cmcc','_jd'))

sum_data = data_res.sum()#添加一行总计
sum_data.name='总计'
data_res = data_res.append(sum_data)

data_res[lv_cmcc] = data_res[cgi]/data_res[gongcan_cgi]
data_res[lv_un] = data_res[gongcan_cgi+'_un']/data_res[cgi]
data_res[lv_te] = data_res[gongcan_cgi+'_te']/data_res[cgi]

data_res.columns = [gongcan_cgi_sum,cgi_sum,un_valid_sum,te_valid_sum,lv_cmcc,lv_un,lv_te]

## 模块3、生成结果

In [ ]:
#定义结果的列
gongcan_cgi_sum,cgi_sum,un_valid_sum,te_valid_sum,lv_cmcc,lv_un,lv_te=[
    '现网小区数量', '软采数量', '联通有效小区', '电信有效小区','软采采集率_本网', '软采小区采集率_联通', '软采小区采集率_电信']

In [ ]:
def sort_data(df,cj='xxx'):
    df_head = df.iloc[:-1]
    df_head_sort = df_head.sort_values(cj,ascending=False)
    df_head_foot = df_head_sort.append(df.iloc[-1])
    for xx in [lv_cmcc,lv_un,lv_te]:
        df_head_foot[xx] = df_head_foot[xx].apply(lambda x: format(x, '.2%'))#转成百分比
    return df_head_foot

In [ ]:
cmcc_res = sort_data(data_res,lv_cmcc)[[gongcan_cgi_sum,cgi_sum,lv_cmcc]].reset_index()
un_res = sort_data(data_res,lv_un)[[gongcan_cgi_sum,cgi_sum,un_valid_sum,lv_un]].reset_index()
te_res = sort_data(data_res,lv_te)[[gongcan_cgi_sum,cgi_sum,te_valid_sum,lv_te]].reset_index()

In [ ]:
cmcc_res.to_csv('G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/cmcc_res.csv',encoding='gbk',index=False)
un_res.to_csv('G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/un_res.csv',encoding='gbk',index=False)
te_res.to_csv('G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/te_res.csv',encoding='gbk',index=False)

# -------------------栅格数据完整性评估----------------------
###   ：使用亿阳导出的栅格数据与理论的栅格数据进行对比，计算栅格采集率

In [ ]:
import pandas as pd
import geopandas as gpd
import mytools

## 模块1、导入栅格数据并与处理

In [ ]:
path = 'G:/1-规划/图层/栅格/2019年/2019年12月栅格/12月栅格原始数据/12011207grid/'
path_2 = 'G:/1-规划/图层/栅格/2019年/2019年12月栅格/11月栅格原始数据/11011110grid/'

f = mytools.othern.file_name_paths(path)
f_2 = mytools.othern.file_name_paths(path_2)

lon,lat = ['lon','lat']

dya,city,grid_id,leftbottom_lon,left_bottom_lat,righttop_lon,righttop_lat,rsrp_count,rsrp_weak,un_count,un_weak,te_count,te_weak=[
    'a.day', 'a.region_name', 'a.grid_id', 'a.grid_leftbottom_longitude','a.grid_leftbottom_latitude', 
    'a.grid_righttop_longitude' ,'a.grid_righttop_latitude','a.rsrp_sample_count', 'a.rsrp_weak', 
    'a.un_maxrsrp_sample_count',  'a.un_maxrsrp_weak_l2', 'a.te_maxrsrp_sample_count','a.te_maxrsrp_weak_l2',]

def data_processing(f):
    data = pd.DataFrame()
    for name in f:
        data_t = pd.read_csv(open(name,encoding='utf-8'),nrows=None,sep='\t')
        data = data.append(data_t)
    data_t_use = data[[dya,city,grid_id,leftbottom_lon,left_bottom_lat,righttop_lon,righttop_lat,rsrp_count,
                       rsrp_weak,un_count,un_weak,te_count,te_weak]]
    data_t_use.loc[:,lon] = (data_t_use[leftbottom_lon]+data_t_use[righttop_lon])/2
    data_t_use.loc[:,lat] = (data_t_use[left_bottom_lat]+data_t_use[righttop_lat])/2
    data_t_use_v1 = data_t_use[[dya,city,grid_id,lon,lat,rsrp_count,rsrp_weak,un_count,un_weak,te_count,te_weak]]
    return data_t_use_v1

grid_1 = data_processing(f)
grid_2 = data_processing(f_2)

## 模块2、整理栅格数据

In [ ]:
res_file_paths = 'g:/1-规划/评估/软采评估/2019第四季度/栅格/1千500万个栅格11和12月汇总.csv'
res_drop_file_paths = 'g:/1-规划/评估/软采评估/2019第四季度/栅格/1千500万个栅格11和12月汇总_去重.csv'

grid_sum = pd.concat([grid_1,grid_2])

grid_sum_qc = grid_sum.sort_values(rsrp_count,ascending=False).drop_duplicates(grid_id)

def add_changjing(data,number=1000000):
    data_t_use_v3 = pd.DataFrame()
    for grind_t in mytools.othern.df_yield(data,number=number):
        data_t_use_v2 = mytools.gisn.points_coverage_merge(grind_t,lon,lat)
        data_t_use_v3 = data_t_use_v3.append(data_t_use_v2)
    return data_t_use_v3

grid_changjing = add_changjing(grid_sum_qc)

# grid_changjing.to_csv(res_file_paths,encoding='utf-8',index=False)

grid_changjing.to_csv(res_drop_file_paths,encoding='utf-8',index=False)

## 模块3、输出结果数据

In [ ]:
grid_changjing = pd.read_csv(open('g:/1-规划/评估/软采评估/2019第四季度/栅格/1千500万个栅格11和12月汇总_去重.csv',encoding='utf-8'))

wuhan_xiangzhen = grid_changjing.loc[(grid_changjing['a.region_name']=='武汉')&(grid_changjing['区域类型']=='乡镇')]

wuhan_xiangzhen.to_clipboard()

grid_paths = 'G:/1-规划/评估/软采评估/2019第四季度/完整性评估/基础文件储存/栅格数据/grid_res.xlsx'

In [ ]:
def data_res(grid_2):
    grid_1 = grid_2.sort_values(rsrp_count,ascending=False).drop_duplicates(grid_id)
    grid_hb1 = grid_1.groupby(city).agg({grid_id:'count'})
    grid_hb2 = grid_1.loc[grid_1[rsrp_count]>100].groupby(city).agg({grid_id:'count'})
    grid_hb = grid_hb1.merge(grid_hb2,how='left',left_index=True,right_index=True,suffixes=('_count', '_yx_count'))
    sum_data = grid_hb.sum()#添加一行总计
    sum_data.name='全网'
    grid_hb = grid_hb.append(sum_data).reset_index()
    grid_hb.columns=['地市','栅格采集数','有效栅格采集数']
    res_dict = dict()
    res_dict['全网'] = grid_hb
    for name_changjing in list(set(grid_1['区域类型'])):
        grid_z_count = grid_1.loc[grid_1['区域类型']==name_changjing].groupby(city).agg({grid_id:'count'})
        grid_z_youxiao_count = grid_1.loc[(grid_1['区域类型']==name_changjing)& (grid_1[rsrp_count]>100)].groupby(city).agg({grid_id:'count'})
        data_z_merge = grid_z_count.merge(grid_z_youxiao_count,how='left',left_index=True,right_index=True,suffixes=('_count', '_yx_count'))
        sum_data = data_z_merge.sum()#添加一行总计
        sum_data.name='全网'
        data_z_merge = data_z_merge.append(sum_data).reset_index()
        data_z_merge.columns=['地市','栅格采集数','有效栅格采集数']
        res_dict[name_changjing] = data_z_merge
    return res_dict

In [ ]:
res_dict = data_res(grid_changjing)

In [ ]:
res_dict['z'] = res_dict.pop('全网')
res_dict['xz'] = res_dict.pop('乡镇')
res_dict['yb'] = res_dict.pop('一般城区')
res_dict['zc'] = res_dict.pop('主城区')
res_dict['xc'] = res_dict.pop('县城')
res_dict['nc'] = res_dict.pop('农村')

In [ ]:
with pd.ExcelWriter(grid_paths) as writer:
    for name in res_dict:
        res_dict[name].to_excel(writer,encoding='gbk',index=False,sheet_name=name)
writer.save()

# -------------------生成word报告并导出----------------------
###   ：使用以上软采小区评估结果、栅格评估结果输出规定的word报告

In [ ]:
import mytools
from docx import Document
import pandas as pd
w = mytools.wordn

In [ ]:
cmcc_res = pd.read_csv(open('G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/cmcc_res.csv',encoding='gbk'))
un_res = pd.read_csv(open('G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/un_res.csv',encoding='gbk'))
te_res = pd.read_csv(open('G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/te_res.csv',encoding='gbk'))

In [ ]:
sheet_new = pd.read_excel('G:/1-规划/评估/软采评估/2019第四季度/完整性评估/基础文件储存/栅格数据/grid_res.xlsx', sheet_name=None)

In [ ]:
sheet = pd.read_excel('G:/1-规划/评估/软采评估/2019第四季度/栅格/栅格采集率跟踪表.xlsx', sheet_name=None)

In [ ]:
max_caiji= dict()
max_caijilv= dict()
for name in sheet:
    data_t = pd.concat([sheet[name].iloc[:,0:3],sheet[name].iloc[:,5:]],axis=1)
    data_t_merge=data_t.merge(sheet_new[name],how='left',on='地市')
    data_t_merge['栅格采集率-201912'] = data_t_merge['栅格采集数']/data_t_merge['栅格数']
    data_t_merge['有效栅格采集率-201912'] = data_t_merge['有效栅格采集数']/data_t_merge['栅格数']
    data_t_merge_res = data_t_merge.drop(columns=['栅格采集数','有效栅格采集数'])
    sheet[name] = mytools.othern.df_sort(data_t_merge_res,data_t_merge_res.columns[3:],'有效栅格采集率-201912')
    max_caiji[name] = sheet[name].loc[sheet[name]['地市']=='全网',[
        '栅格采集率-201906','栅格采集率-201909','栅格采集率-201912']].replace(regex=[r'\%|'],value='').astype('float').iloc[0].max()
    max_caijilv[name] = sheet[name].loc[sheet[name]['地市']=='全网',[
        '有效栅格采集率-201906','有效栅格采集率-201909','有效栅格采集率-201912']].replace(regex=[r'\%|'],value='').astype('float').iloc[0].max()

In [ ]:
grid_lie_name = ['地市', '场景', '栅格数','栅格采集率-201906','有效栅格采集率-201906',
                 '栅格采集率-201909', '有效栅格采集率-201909', '栅格采集率-201912','有效栅格采集率-201912']

In [ ]:
document = Document()

In [ ]:
w.add_heading(document,'MR软采采集评估报告',jibie=1,size=18,duiqi='c')
w.add_heading(document,'1.软采采集率',jibie=2,size=16)

w.add_paragraph(document,'软采采集率评估旨在软采采集的完整性。将在每次软采开启后，针对软采采集的数据分小区和栅格维度进行完整性评估，以保证全网覆盖评估的准确性。')
w.add_heading(document,'1.1.软采小区采集率',jibie=3,size=14)
w.add_paragraph(document,'软采小区涉及到本网和竞对两部分。如未要求开启竞对信息，则仅考虑本网软采小区采集率。')
w.add_heading(document,'1.1.1.软采小区采集率-本网',jibie=4,size=12)
p = document.add_paragraph('')
w.add_text(p,'定义：',bold=True)
w.add_text(p,'软采采集率_本网 = 软采数量（小区总采样点>0）/总小区数。注：软采采集到的小区采样点均大于0。')
p = document.add_paragraph('')
w.add_text(p,'完整性门限：',bold=True)
w.add_text(p,'建议本网软采小区采集率高于')
w.add_text(p,'80%。',bold=True,color='r')
w.add_paragraph(document,'本次12月软采小区采集率如下：')
w.add_df(document,cmcc_res,'m')
p = document.add_paragraph('')

w.add_heading(document,'1.1.2.软采小区采集率-联通',jibie=4,size=12)
p = document.add_paragraph('')
w.add_text(p,'定义：',bold=True)
w.add_text(p,'联通有效小区/软采数量')
w.add_text(p,'完整性门限：',bold=True)
w.add_text(p,'建议本网软采小区采集率高于')
w.add_text(p,'80%。',bold=True,color='r')
w.add_paragraph(document,'本次12月软采小区采集率如下：')
w.add_df(document,un_res,'m')
p = document.add_paragraph('')

w.add_heading(document,'1.1.3.软采小区采集率-电信',jibie=4,size=12)
p = document.add_paragraph('')
w.add_text(p,'定义：',bold=True)
w.add_text(p,'电信有效小区/电信数量')
w.add_text(p,'完整性门限：',bold=True)
w.add_text(p,'建议本网软采小区采集率高于')
w.add_text(p,'80%。',bold=True,color='r')
w.add_paragraph(document,'本次12月软采小区采集率如下：')
w.add_df(document,te_res,'m')
p = document.add_paragraph('')

w.add_heading(document,'1.2.软采栅格采集率',jibie=3,size=14)
w.add_paragraph(document,'软采栅格当前亿阳无法全量栅格数据。')
w.add_paragraph(document,'全量栅格数据拟采用湖北省面积除以栅格面积得到软采栅格数量。')
w.add_paragraph(document,'栅格采集率=采集到的栅格数/全量栅格数据。')
w.add_paragraph(document,'有效栅格采集率=采集到的有效栅格数（栅格采样点>100）/全量栅格数据。')

w.add_heading(document,'1.2.1.总体栅格采集率',jibie=4,size=12)
w.add_paragraph(document,'对比2019年6月、2019年9月、2019年12月三次软采栅格的数据情况，栅格采集率最 \
高为{caiji}%,有效栅格采集率最高在{youxiaocaiji}%。'.format(caiji=max_caiji['z'],youxiaocaiji=max_caijilv['z']))
p = document.add_paragraph('')
w.add_text(p,'完整性门限：',bold=True)
w.add_text(p,'建议栅格采集率高于')
w.add_text(p,'50%。',bold=True,color='r')
w.add_df(document,sheet['z'][grid_lie_name],'m',size=10)
p = document.add_paragraph('')

bianliang='zc'
w.add_heading(document,'1.2.2.总体栅格采集率',jibie=4,size=12)
w.add_paragraph(document,'分场景评估栅格采集情况：')
p = document.add_paragraph('')
w.add_text(p,'主城区：',bold=True)
w.add_text(p,'主城区采集率较高，栅格采集率最高为{caiji}%,有效栅格采集率最高在 \
{youxiaocaiji}%。'.format(caiji=max_caiji[bianliang],youxiaocaiji=max_caijilv[bianliang]))
p = document.add_paragraph('')
w.add_text(p,'完整性门限：',bold=True)
w.add_text(p,'建议主城区栅格采集率高于')
w.add_text(p,'90%。',bold=True,color='r')
w.add_df(document,sheet[bianliang][grid_lie_name],'m',size=10)
p = document.add_paragraph('')

bianliang='yb'
p = document.add_paragraph('')
w.add_text(p,'一般城区：',bold=True)
w.add_text(p,'一般城区采集率较高，栅格采集率最高为{caiji}%,有效栅格采集率最高在 \
{youxiaocaiji}%。'.format(caiji=max_caiji[bianliang],youxiaocaiji=max_caijilv[bianliang]))
p = document.add_paragraph('')
w.add_text(p,'完整性门限：',bold=True)
w.add_text(p,'建议一般城区栅格采集率高于')
w.add_text(p,'80%。',bold=True,color='r')
w.add_df(document,sheet[bianliang][grid_lie_name],'m',size=9)
p = document.add_paragraph('')

bianliang='xc'
p = document.add_paragraph('')
w.add_text(p,'县城：',bold=True)
w.add_text(p,'县城采集率较高，栅格采集率最高为{caiji}%,有效栅格采集率最高在 \
{youxiaocaiji}%。'.format(caiji=max_caiji[bianliang],youxiaocaiji=max_caijilv[bianliang]))
p = document.add_paragraph('')
w.add_text(p,'完整性门限：',bold=True)
w.add_text(p,'建议县城栅格采集率高于')
w.add_text(p,'70%。',bold=True,color='r')
w.add_df(document,sheet[bianliang][grid_lie_name],'m',size=10)
p = document.add_paragraph('')

bianliang='xz'
p = document.add_paragraph('')
w.add_text(p,'乡镇：',bold=True)
w.add_text(p,'乡镇采集率较高，栅格采集率最高为{caiji}%,有效栅格采集率最高在 \
{youxiaocaiji}%。'.format(caiji=max_caiji[bianliang],youxiaocaiji=max_caijilv[bianliang]))
p = document.add_paragraph('')
w.add_text(p,'完整性门限：',bold=True)
w.add_text(p,'建议乡镇栅格采集率高于')
w.add_text(p,'50%。',bold=True,color='r')
w.add_df(document,sheet[bianliang][grid_lie_name],'m',size=10)
p = document.add_paragraph('')


bianliang='nc'
p = document.add_paragraph('')
w.add_text(p,'农村：',bold=True)
w.add_text(p,'农村采集率较高，栅格采集率最高为{caiji}%,有效栅格采集率最高在 \
{youxiaocaiji}%。'.format(caiji=max_caiji[bianliang],youxiaocaiji=max_caijilv[bianliang]))
p = document.add_paragraph('')
w.add_text(p,'完整性门限：',bold=True)
w.add_text(p,'建议农村栅格采集率高于')
w.add_text(p,'10%。',bold=True,color='r')
w.add_df(document,sheet[bianliang][grid_lie_name],'m',size=10)
p = document.add_paragraph('')

document.save('G:/1-规划/MRO/1、原始数据/摸底测试相关/2019年/12月/mro/四季度评估使用/程序储存数据/四季度完整性评估报告.docx')
print('ok')

# --------------------------------------------------------------------